In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
import pickle
import networkx as nx
from multiprocessing import Pool
from load_files_pop import (
    Ref,
    Z_20,
    Ref_pop20,
    Z_200,
    Ref_pop200,
    Z_2000,
    Ref_pop2000
)
from Reference_pop import simulate_reference_time_series
from plot_search_results import (
    plot_pareto_front3D,
    plot_pulse_ensemble_time_series,
    plot_ref_ensemble_time_series
)
from define_circuit import Topo
from amplifier_problem import Amplifier
from signal_conditioner_problem import SignalConditioner

orange_ = [i/255 for i in [230, 159, 0]]
sky_blue = [i/255 for i in [86, 180, 233]]
pink_ = [i/255 for i in [204, 121, 167]]
bluish_green = [i/255 for i in [0, 158, 115]]
vermillion = [i/255 for i in [213, 94, 0]]
yellow_ = [i/255 for i in [240, 228, 66]]
blue_ = [i/255 for i in [0, 114, 178]]

colors = [orange_, sky_blue, pink_, bluish_green, vermillion, yellow_, blue_]

## Amplifier Z matrix sampling all circuits vs. top pct

In [ ]:
plt.figure(figsize=(3, 3))
path_Z_sample = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/Amp_seed_pop_vary_dose/Original_hyperparams_worked_well/2024-04-23_Amplifier_pop_vary_dose_original_hp_seed_0/Amplifier_pop_vary_dose_Z_matrix_sampling_all/Z_matrix_sampling_for_CI.pkl"
with open(path_Z_sample, "rb") as fid:
    Z_sample = pickle.load(fid)
print(np.mean(Z_sample["ON_rel_std_error"]))
# print(max(Z_sample["ON_rel_std_error"])*0.5)
plt.hist(Z_sample["ON_rel_std_error"], bins=50)
# Z_sample["ON_rel_std_error"].plot.density(ind=1000)
plt.axvline(x=np.mean(Z_sample["ON_rel_std_error"]), label="mean", linestyle="dotted", color="k", linewidth=1.5)
plt.xlim([0, 0.85])
plt.xlabel("ON_rel std error")
plt.ylabel("Count")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(3, 3))
path_Z_sample2 = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/Amp_seed_pop_vary_dose/Original_hyperparams_worked_well/2024-04-23_Amplifier_pop_vary_dose_original_hp_seed_0/2024-04-24_Amplifier_pop_vary_dose_Z_matrix_sampling/Z_matrix_sampling_for_CI.pkl"
with open(path_Z_sample2, "rb") as fid:
    Z_sample2 = pickle.load(fid)
# print(max(Z_sample2["ON_rel_std_error"]))
plt.hist(Z_sample2["ON_rel_std_error"], bins=20)
plt.axvline(x=max(Z_sample2["ON_rel_std_error"]), label="CI value", linestyle="dotted", color="k", linewidth=1.5)
plt.xlim([0, 0.85])
plt.xlabel("ON_rel std error")
plt.ylabel("Count")
plt.legend()
plt.show()

## Amplifier CI circuit comparisons: 20 cell vs 200 cell vs 2000 cell

In [ ]:
path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/GA_results/Amp_seed_pop_vary_dose/Original_hyperparams_worked_well/2024-04-23_Amplifier_pop_vary_dose_original_hp_seed_0/2024-08-23_results_analysis_full_CI/selected_results_ON_rel_full_CI.pkl"
amp_CI = pd.read_pickle(path)
all_ON_rel_20 = amp_CI["ON_rel"].tolist()
all_ON_rel_200 = []
all_ON_rel_2000 = []
for index, row in amp_CI.iterrows():
    print("20 cell: ", all_ON_rel_20[index])
    topo = amp_CI.at[index, "Topology"]
    amp_problem_200 = Amplifier(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                           DsRed_inhibitor=False, num_dict={}, n_gen=None, probability_crossover=None, 
                           probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200, num_processes=8, max_time=42, single_cell_tracking=True)
    [ON_rel_200, all_cells_dict_200] = amp_problem_200.func(topo)
    print("200 cell: ", ON_rel_200)
    all_ON_rel_200.append(ON_rel_200)

    amp_problem_2000 = Amplifier(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                           DsRed_inhibitor=False, num_dict={}, n_gen=None, probability_crossover=None, 
                           probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_2000, num_processes=8, max_time=42, single_cell_tracking=True)
    [ON_rel_2000, all_cells_dict_2000] = amp_problem_2000.func(topo)
    print("2000 cell: ", ON_rel_2000)
    all_ON_rel_2000.append(ON_rel_2000)
    print("***** topology ", index, " done *****")

In [4]:
amp_pop_comparison = {"20_cell": all_ON_rel_20, "200_cell": all_ON_rel_200, "2000_cell": all_ON_rel_2000}
amp_pop_df = pd.DataFrame(amp_pop_comparison)
amp_pop_df_pos = amp_pop_df.abs()
path_save = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/GA_results/Amp_seed_pop_vary_dose/Original_hyperparams_worked_well/2024-04-23_Amplifier_pop_vary_dose_original_hp_seed_0/2024-08-23_results_analysis_full_CI/"
amp_pop_df_pos.to_csv(path_save+"pop_model_comparisons.csv")
x_vals = [0]*len(amp_pop_df_pos)
jittered_x = x_vals + np.random.rand(
    len(x_vals))
amp_pop_df_pos["jittered_x"] = jittered_x
# display(amp_pop_df_pos)

In [ ]:
path_save = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/GA_results/Amp_seed_pop_vary_dose/Original_hyperparams_worked_well/2024-04-23_Amplifier_pop_vary_dose_original_hp_seed_0/2024-08-23_results_analysis_full_CI/"

plt.figure(figsize=(3,3))
plt.plot(amp_pop_df_pos["jittered_x"], amp_pop_df_pos["20_cell"], label="20 cell model", linestyle="none", marker="o", markersize="4", zorder=1)
plt.plot(amp_pop_df_pos["jittered_x"], amp_pop_df_pos["200_cell"], label="200 cell model", linestyle="none", marker="o", markersize="4", zorder=2)
# plt.plot(amp_pop_df_pos["jittered_x"], amp_pop_df_pos["2000_cell"], label="2000 cell model", linestyle="none", marker="o", markersize="4")
lower_bound = [63.11786016]*len(amp_pop_df_pos)
upper_bound = [63.50911412131601]*len(amp_pop_df_pos)
jittered_x.sort()
plt.fill_between(jittered_x, lower_bound, upper_bound, alpha=0.3, color="grey", zorder=3)
plt.xticks([])
plt.legend()
# plt.show()
plt.savefig(path_save+"20vs200_cell_CI.svg")

## Signal conditioner pareto circuit comparisons: 20 cell vs 200 cell vs 2000 cell

In [ ]:
from signal_conditioner_problem import SignalConditioner
path_sc = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/GA_results/SC_seed_pop_DsRED_inhibitor/Original_hyperparams/SC_original_hp_seed_0/2024-09-05_results_analysis/selected_results_full_pareto_front.pkl"
sc_pareto = pd.read_pickle(path_sc)
# display(sc_pareto)

all_ON_rel_20 = sc_pareto["ON_rel"].tolist()
all_FI_rel_20 = sc_pareto["FI_rel"].tolist()
# num_topos = len(all_ON_rel_20)

all_ON_rel_200 = []
all_FI_rel_200 = []

# all_ON_rel_2000 = []
# all_FI_rel_2000 = []

for index, row in sc_pareto.iterrows():
    print("20 cell ON_rel, FI_rel: ", all_ON_rel_20[index], all_FI_rel_20[index])
    topo = sc_pareto.at[index, "Topology"]
    sc_problem_200 = SignalConditioner(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                           DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                           probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200, num_processes=8, max_time=42, single_cell_tracking=True)
    [[ON_rel_200, FI_rel_200], all_cells_dict_200] = sc_problem_200.func(topo)
    print("200 cell ON_rel, FI_rel: ", ON_rel_200, FI_rel_200)
    all_ON_rel_200.append(ON_rel_200)
    all_FI_rel_200.append(FI_rel_200)

    # sc_problem_2000 = SignalConditioner(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
    #                        DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
    #                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_2000, num_processes=8, max_time=42, single_cell_tracking=True)
    # [[ON_rel_2000, FI_rel_2000], all_cells_dict_2000] = sc_problem_2000.func(topo)
    # print("2000 cell ON_rel, FI_rel: ", ON_rel_2000, FI_rel_2000)
    # all_ON_rel_2000.append(ON_rel_2000)
    # all_FI_rel_2000.append(FI_rel_2000)    
    # print("***** topology ", index, " done *****")
    # topo_num = index + 1
    # if topo_num == num_topos:
    #     break

sc_pop_comparison = {"20_cell_ON_rel": all_ON_rel_20, "20_cell_FI_rel": all_FI_rel_20,
                     "200_cell_ON_rel": all_ON_rel_200, "200_cell_FI_rel": all_FI_rel_200,
                    #  "2000_cell_ON_rel": all_ON_rel_2000, "2000_cell_FI_rel": all_FI_rel_2000,
}
sc_pop_df = pd.DataFrame(sc_pop_comparison)
sc_pop_df_pos = sc_pop_df.abs()
path_save = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/GA_results/SC_seed_pop_DsRED_inhibitor/Original_hyperparams/SC_original_hp_seed_0/2024-09-05_results_analysis/"
# sc_pop_df_pos.to_csv(path_save+"pop_model_comparisons_20v200.csv")

In [ ]:
results_path = "/Users/kdreyer/Documents/Github/GraphGA/" + "GA_results/SC_seed_pop_DsRED_inhibitor/Original_hyperparams/2024-05-02_Signal_Cond_pop_DsRED_inhibitor_original_hp_seed_0/"
obj_labels = ["ON_rel", "FI_rel"]
objectives_fname = "final_objectives_df.pkl"
pareto_front_objectives = pd.read_pickle(
    results_path+objectives_fname
)
selected_objectives = (
    pareto_front_objectives.drop_duplicates()
)
CI_metric_maxes = [0.7549563624285208, 0.004862470888440853]
obj1_CI = CI_metric_maxes[0]
obj2_CI = CI_metric_maxes[1]

upper_obj1 = np.abs(np.array(selected_objectives[obj_labels[0]]))
sorted_upper_idx = np.argsort(upper_obj1)
sorted_upper_obj1 = upper_obj1[sorted_upper_idx]

upper_obj1_plus = np.array([i+obj1_CI for i in upper_obj1])
sorted_upper_plus_idx = np.argsort(upper_obj1_plus)
sorted_upper_obj1_plus = upper_obj1_plus[sorted_upper_plus_idx]

lower_obj1 = np.array([i-obj1_CI for i in upper_obj1])
sorted_lower_idx = np.argsort(lower_obj1)
sorted_lower_obj1 = lower_obj1[sorted_lower_idx]

# upper_obj2 = np.array([i+obj2_CI_list[0] for i in (objectives[obj_labels[1]]*-1)])
upper_obj2 = np.abs(np.array(selected_objectives[obj_labels[1]]))
sorted_upper_obj2 = upper_obj2[sorted_upper_idx]

upper_obj2_plus = np.array([i+obj2_CI for i in upper_obj2])
sorted_upper_obj2_plus = upper_obj2_plus[sorted_upper_plus_idx]

lower_obj2 = np.array([i-obj2_CI for i in upper_obj2])
sorted_lower_obj2 = lower_obj2[sorted_lower_idx]

xfill = np.sort(np.concatenate([upper_obj1, lower_obj1]))
xfill_plus = np.sort(np.concatenate([upper_obj1_plus, lower_obj1]))

y1fill = np.interp(xfill, sorted_upper_obj1, sorted_upper_obj2)
y2fill = np.interp(xfill, sorted_lower_obj1, sorted_lower_obj2)
y2_plus_fill = np.interp(xfill_plus, sorted_lower_obj1, sorted_lower_obj2)
y1_plus_fill = np.interp(xfill_plus, sorted_upper_obj1_plus, sorted_upper_obj2_plus)
plt.figure(figsize=(5,5))
# plt.fill_between(xfill, y1fill, y2fill, alpha=0.3, color="grey", zorder=3)
plt.fill_between(xfill_plus, y1_plus_fill, y2_plus_fill, alpha=0.1, color="blue", zorder=1)
plt.plot(sc_pop_df_pos["20_cell_ON_rel"], sc_pop_df_pos["20_cell_FI_rel"], label="20 cell model", linestyle="none", marker="o", markersize="2", zorder=0)
plt.plot(sc_pop_df_pos["200_cell_ON_rel"], sc_pop_df_pos["200_cell_FI_rel"], label="200 cell model", linestyle="none", marker="o", markersize="2", zorder=1)
# plt.plot(sc_pop_df_pos["2000_cell_ON_rel"], sc_pop_df_pos["2000_cell_FI_rel"], label="2000 cell model", linestyle="none", marker="o", markersize="2")
# plt.legend()
# plt.xlim(left=40)
# plt.ylim(top=0.6)
# ax.plot(all_objectives[:, 0], all_objectives[:, 1], linestyle="none", 
#         marker="o", markersize=1, color="black", zorder=1)
plt.xlabel(obj_labels[0])
plt.ylabel(obj_labels[1])
plt.show()
# plt.savefig(results_path+"20vs200_cell_CI.svg", bbox_inches="tight")

In [ ]:
# all_ON_rel.sort()
# print(all_ON_rel)
#     if "Z2" in amp_CI.at[index, "Doses"].keys() and "Z6" in amp_CI.at[index, "Doses"].keys():
#         if len(amp_CI.at[index, "Edge list"]) == 10:
#             opt_topo_idx.append(index)
#             opt_topo_doses.append(amp_CI.at[index, "Doses"])
#         elif len(amp_CI.at[index, "Edge list"]) == 9:
#             if ("Z2", "Z2") in amp_CI.at[index, "Edge list"] and ("Z2", "Rep") in amp_CI.at[index, "Edge list"]:
#                 # print(amp_CI.at[index, "Edge list"])
#                 green_9_edge.append(index)
#             elif ("Z2", "Z2") in amp_CI.at[index, "Edge list"] and ("Z2", "Rep") not in amp_CI.at[index, "Edge list"]:
#                 # print(amp_CI.at[index, "Edge list"])
#                 yellow_9_edge.append(index)
#             # elif ("Z2", "Z2") not in amp_CI.at[index, "Edge list"]:
#             else:
#                 orange_9_edge.append(index)
#         elif len(amp_CI.at[index, "Edge list"]) == 8:
#             if ("Z2", "Z2") not in amp_CI.at[index, "Edge list"]:
#                 pink_8_edge.append(index)
#             else:
#                 purple_8_edge.append(index)        
#     else:
#         other_ZFs_idx.append(index)
#         print(index, amp_CI.at[index, "Edge list"])
# # print(len(opt_topo_doses))
# print(other_ZFs_idx)

In [ ]:
path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/GA_results/Amp_seed_pop_vary_dose/Original_hyperparams_worked_well/2024-04-23_Amplifier_pop_vary_dose_original_hp_seed_0/unique_objectives.pkl"

unique_objs = pd.read_pickle(path).flatten()
sorted_objs = np.abs(np.sort(unique_objs))
# print(sorted_objs)
gte_2pct = [i for i in sorted_objs if i >= 62.23882]
print(gte_2pct)

In [ ]:
path_obj = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/GA_results/SC_seed_single_cell_DsRED_inhibitor/Original_hyperparams/SC_original_hp_seed_3/final_objectives_df.pkl"
path_circuit = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/GA_results/SC_seed_single_cell_DsRED_inhibitor/Original_hyperparams/SC_original_hp_seed_3/final_population.pkl"

objs = pd.read_pickle(path_obj)
circuits = pd.read_pickle(path_circuit)
# circuits[1][0].plot_graph()
# display(objs)
display(objs[objs["type"] == "Inhibitors"])

## Amplifier and signal conditioner pre GA 20v200 cell model comparison (using single cell results)

### Amplifier

In [9]:
amp_single_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Amplifier_single_cell/Original_hyperparams_vary_dose/Amplifier_single_cell_vary_dose_original_hp_seed_0/"
obj_fname = "all_unique_obj.pkl"
circuit_fname = "all_unique_circuits.pkl"

with open(amp_single_path+obj_fname, "rb") as fid:
    amp_single_obj = pickle.load(fid)
amp_single_obj = abs(amp_single_obj.flatten())
with open(amp_single_path+circuit_fname, "rb") as fid:
    amp_single_circuits = pickle.load(fid)
amp_single_circuits = amp_single_circuits.flatten()

selected_obj_idx_high = np.where(
    amp_single_obj > 63
)[0]
selected_unique_obj_high = amp_single_obj[selected_obj_idx_high]
# print(len(selected_unique_obj_high))
selected_unique_circuits_high = amp_single_circuits[selected_obj_idx_high]


selected_obj_idx_mid = np.where(
    np.logical_and(
        amp_single_obj > 37, 
        amp_single_obj < 42))[0]
selected_unique_obj_mid = amp_single_obj[selected_obj_idx_mid]
selected_unique_circuits_mid = amp_single_circuits[selected_obj_idx_mid]
# print(len(selected_unique_obj_mid))

selected_obj_idx_low = np.where(
        amp_single_obj < 20
        )[0]
selected_unique_obj_low = amp_single_obj[selected_obj_idx_low]
selected_unique_circuits_low = amp_single_circuits[selected_obj_idx_low]

# print(len(selected_unique_obj_low))

### high

In [19]:
amp_problem_20 = Amplifier(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=False, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, num_processes=8, max_time=42, single_cell_tracking=False)


amp_problem_200 = Amplifier(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=False, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200, num_processes=8, max_time=42, single_cell_tracking=False)

# if __name__ == "__main__":
#     with Pool(8) as pool:
#         all_ON_rel_20 = pool.imap(amp_problem_20.func, selected_unique_circuits_high)
#         pool.close()
#         pool.join()
#     all_ON_rel_20 = list(all_ON_rel_20)
#     all_ON_rel_20 = np.asarray(all_ON_rel_20)

#     with Pool(8) as pool:
#         all_ON_rel_200 = pool.imap(amp_problem_200.func, selected_unique_circuits_high)
#         pool.close()
#         pool.join()
#     all_ON_rel_200 = list(all_ON_rel_200)
#     all_ON_rel_200 = np.asarray(all_ON_rel_200)


# amp_pop_comparison = {"20_cell": all_ON_rel_20, "200_cell": all_ON_rel_200}
# amp_pop_df = pd.DataFrame(amp_pop_comparison)
# amp_pop_df_pos = amp_pop_df.abs()
# path_save = amp_single_path + "pop20v200_comparison_high/"
# amp_pop_df_pos.to_csv(path_save+"pop_model_comparisons.csv")

### mid-range

In [20]:
amp_problem_20 = Amplifier(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=False, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, num_processes=8, max_time=42, single_cell_tracking=False)


amp_problem_200 = Amplifier(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=False, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200, num_processes=8, max_time=42, single_cell_tracking=False)


# if __name__ == "__main__":
#     with Pool(8) as pool:
#         all_ON_rel_20 = pool.imap(amp_problem_20.func, selected_unique_circuits_mid)
#         pool.close()
#         pool.join()
#     all_ON_rel_20 = list(all_ON_rel_20)
#     all_ON_rel_20 = np.asarray(all_ON_rel_20)

#     with Pool(8) as pool:
#         all_ON_rel_200 = pool.imap(amp_problem_200.func, selected_unique_circuits_mid)
#         pool.close()
#         pool.join()
#     all_ON_rel_200 = list(all_ON_rel_200)
#     all_ON_rel_200 = np.asarray(all_ON_rel_200)


# amp_pop_comparison = {"20_cell": all_ON_rel_20, "200_cell": all_ON_rel_200}
# amp_pop_df = pd.DataFrame(amp_pop_comparison)
# amp_pop_df_pos = amp_pop_df.abs()
# path_save = amp_single_path + "pop20v200_comparison_mid-range/"
# amp_pop_df_pos.to_csv(path_save+"pop_model_comparisons.csv")

### low

In [21]:
amp_problem_20 = Amplifier(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=False, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, num_processes=8, max_time=42, single_cell_tracking=False)


amp_problem_200 = Amplifier(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=False, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200, num_processes=8, max_time=42, single_cell_tracking=False)


# if __name__ == "__main__":
#     with Pool(8) as pool:
#         all_ON_rel_20 = pool.imap(amp_problem_20.func, selected_unique_circuits_low)
#         pool.close()
#         pool.join()
#     all_ON_rel_20 = list(all_ON_rel_20)
#     all_ON_rel_20 = np.asarray(all_ON_rel_20)

#     with Pool(8) as pool:
#         all_ON_rel_200 = pool.imap(amp_problem_200.func, selected_unique_circuits_low)
#         pool.close()
#         pool.join()
#     all_ON_rel_200 = list(all_ON_rel_200)
#     all_ON_rel_200 = np.asarray(all_ON_rel_200)


# amp_pop_comparison = {"20_cell": all_ON_rel_20, "200_cell": all_ON_rel_200}
# amp_pop_df = pd.DataFrame(amp_pop_comparison)
# amp_pop_df_pos = amp_pop_df.abs()
# path_save = amp_single_path + "pop20v200_comparison_low/"
# amp_pop_df_pos.to_csv(path_save+"pop_model_comparisons.csv")

### Signal conditioner

In [ ]:
path_unique_objs = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_single_cell/synTF-R-DsR/Optimized_hyperparams_fixed_pop_max_hv_ngen50/run4_ngen120/2024-10-04_Signal_cond_single_DsRED_opt_hps_run4_seed_3/"
fname = "all_unique_obj.pkl"
pareto_fname = "final_objectives_df.pkl"

with open(path_unique_objs+fname, "rb") as fid:
    objs = pickle.load(fid)
# print(len(objs))
pareto_objs = pd.read_pickle(path_unique_objs+pareto_fname)

# plt.plot(abs(objs[:, 0]), abs(objs[:, 1]), linestyle="none", marker="o", markersize="2")
# plt.plot(abs(pareto_objs["ON_rel"]), abs(pareto_objs["FI_rel"]), linestyle="none", marker="o", markersize="2")
# plt.xlim([0, 10])
# plt.axvline(2.5, 0, 1.8, color="grey")
# plt.axhline(1.0, 0, 10)
# plt.axhline(0.75, 0, 10)
# plt.show()

In [ ]:
path_sc = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_single_cell/synTF-R-DsR/Optimized_hyperparams_fixed_pop_max_hv_ngen50/run4_ngen120/2024-10-04_Signal_cond_single_DsRED_opt_hps_run4_seed_3/"
pareto_fname = "final_objectives_df.pkl"
pareto_circuits_fname = "final_population.pkl"
unique_obj_fname = "all_unique_obj.pkl"
unique_circuits_fname = "all_unique_circuits.pkl"

sc_pareto = pd.read_pickle(path_sc+pareto_fname)
# display(sc_pareto)
with open(path_sc+pareto_circuits_fname, "rb") as fid:
    pareto_circuits = pickle.load(fid)
pareto_circuits = pareto_circuits.flatten()
with open(path_sc+unique_obj_fname, "rb") as fid:
    unique_obj = pickle.load(fid)
unique_obj_df = pd.DataFrame(unique_obj, columns=["ON_rel", "FI_rel"])
unique_obj_df = abs(unique_obj_df)
with open(path_sc+unique_circuits_fname, "rb") as fid:
    unique_circuits = pickle.load(fid)
unique_circuits = unique_circuits.flatten()
unique_obj_df["Topology"] = unique_circuits
# display(unique_obj_df)

selected_obj_mid = unique_obj_df[
    (unique_obj_df["ON_rel"] < 2.5) &
    ((unique_obj_df["FI_rel"] > 1.0) & (unique_obj_df["FI_rel"] < 1.5))
]

selected_obj_low = unique_obj_df[
    (unique_obj_df["ON_rel"] < 1.6) &
    ((unique_obj_df["FI_rel"] > 0.75) & (unique_obj_df["FI_rel"] < 1.0))
]
# display(selected_obj_mid)
# print(len(selected_obj_low))
# print(len(selected_obj_mid))
# plt.plot(abs(objs[:, 0]), abs(objs[:, 1]), linestyle="none", marker="o", markersize="2")
# plt.plot(abs(pareto_objs["ON_rel"]), abs(pareto_objs["FI_rel"]), linestyle="none", marker="o", markersize="2")
# plt.plot(selected_obj_low["ON_rel"], selected_obj_low["FI_rel"], linestyle="none", marker="o", markersize="2")
# plt.xlim([0, 10])
# plt.axvline(1.6, 0, 1.8, color="grey")
# plt.axhline(1.5, 0, 10)
# plt.axhline(1, 0, 10)
# plt.show()

### pareto

In [60]:
sc_problem_20 = SignalConditioner(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, num_processes=8, max_time=42, single_cell_tracking=False)

sc_problem_200 = SignalConditioner(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200, num_processes=8, max_time=42, single_cell_tracking=False)


if __name__ == "__main__":
    with Pool(8) as pool:
        all_objs_20 = pool.imap(sc_problem_20.func, pareto_circuits)
        pool.close()
        pool.join()
    all_objs_20 = list(all_objs_20)
    all_objs_20 = np.asarray(all_objs_20)
    all_ON_rel_20 = all_objs_20[:,0]
    all_FI_rel_20 = all_objs_20[:,1]

    with Pool(8) as pool:
        all_objs_200 = pool.imap(sc_problem_200.func, pareto_circuits)
        pool.close()
        pool.join()
    all_objs_200 = list(all_objs_200)
    all_objs_200 = np.asarray(all_objs_200)
    all_ON_rel_200 = all_objs_200[:,0]
    all_FI_rel_200 = all_objs_200[:,1]


sc_pop_comparison = {"20_cell_ON_rel": all_ON_rel_20, "20_cell_FI_rel": all_FI_rel_20,
                    "200_cell_ON_rel": all_ON_rel_200, "200_cell_FI_rel": all_FI_rel_200,
}
sc_pop_df = pd.DataFrame(sc_pop_comparison)
sc_pop_df_pos = sc_pop_df.abs()

path_save = path_sc + "pop20v200_comparison_pareto/"
sc_pop_df_pos.to_csv(path_save+"pop_model_comparisons.csv")

### mid-range

In [62]:
sc_problem_20 = SignalConditioner(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, num_processes=8, max_time=42, single_cell_tracking=False)

sc_problem_200 = SignalConditioner(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200, num_processes=8, max_time=42, single_cell_tracking=False)

topologies = selected_obj_mid["Topology"].tolist()
if __name__ == "__main__":
    with Pool(8) as pool:
        all_objs_20 = pool.imap(sc_problem_20.func, topologies)
        pool.close()
        pool.join()
    all_objs_20 = list(all_objs_20)
    all_objs_20 = np.asarray(all_objs_20)
    all_ON_rel_20 = all_objs_20[:,0]
    all_FI_rel_20 = all_objs_20[:,1]

    with Pool(8) as pool:
        all_objs_200 = pool.imap(sc_problem_200.func, topologies)
        pool.close()
        pool.join()
    all_objs_200 = list(all_objs_200)
    all_objs_200 = np.asarray(all_objs_200)
    all_ON_rel_200 = all_objs_200[:,0]
    all_FI_rel_200 = all_objs_200[:,1]


sc_pop_comparison = {"20_cell_ON_rel": all_ON_rel_20, "20_cell_FI_rel": all_FI_rel_20,
                    "200_cell_ON_rel": all_ON_rel_200, "200_cell_FI_rel": all_FI_rel_200,
}
sc_pop_df = pd.DataFrame(sc_pop_comparison)
sc_pop_df_pos = sc_pop_df.abs()

path_save = path_sc + "pop20v200_comparison_mid-range/"
sc_pop_df_pos.to_csv(path_save+"pop_model_comparisons.csv")

### low

In [63]:
sc_problem_20 = SignalConditioner(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, num_processes=8, max_time=42, single_cell_tracking=False)

sc_problem_200 = SignalConditioner(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200, num_processes=8, max_time=42, single_cell_tracking=False)

topologies = selected_obj_low["Topology"].tolist()
if __name__ == "__main__":
    with Pool(8) as pool:
        all_objs_20 = pool.imap(sc_problem_20.func, topologies)
        pool.close()
        pool.join()
    all_objs_20 = list(all_objs_20)
    all_objs_20 = np.asarray(all_objs_20)
    all_ON_rel_20 = all_objs_20[:,0]
    all_FI_rel_20 = all_objs_20[:,1]

    with Pool(8) as pool:
        all_objs_200 = pool.imap(sc_problem_200.func, topologies)
        pool.close()
        pool.join()
    all_objs_200 = list(all_objs_200)
    all_objs_200 = np.asarray(all_objs_200)
    all_ON_rel_200 = all_objs_200[:,0]
    all_FI_rel_200 = all_objs_200[:,1]


sc_pop_comparison = {"20_cell_ON_rel": all_ON_rel_20, "20_cell_FI_rel": all_FI_rel_20,
                    "200_cell_ON_rel": all_ON_rel_200, "200_cell_FI_rel": all_FI_rel_200,
}
sc_pop_df = pd.DataFrame(sc_pop_comparison)
sc_pop_df_pos = sc_pop_df.abs()

path_save = path_sc + "pop20v200_comparison_low/"
sc_pop_df_pos.to_csv(path_save+"pop_model_comparisons.csv")

## Amplifier and signal conditioner post-facto 20v200 cell model comparison

### Amplifier

In [ ]:
amp_pop_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Amplifier_pop/Original_hyperparams_vary_dose/Amplifier_pop_vary_dose_original_hp_seed_0/"
obj_fname = "unique_objectives.pkl"
circuit_fname = "unique_circuits.pkl"
with open(amp_pop_path+obj_fname, "rb") as fid:
    amp_pop20_obj = pickle.load(fid)
amp_pop20_obj = abs(amp_pop20_obj.flatten())
with open(amp_pop_path+circuit_fname, "rb") as fid:
    amp_pop20_circuits = pickle.load(fid)
amp_pop20_circuits = amp_pop20_circuits.flatten()

selected_obj_idx_mid = np.where(
    np.logical_and(
        amp_pop20_obj > 37, 
        amp_pop20_obj < 42))[0]
selected_unique_obj_mid = amp_pop20_obj[selected_obj_idx_mid]
selected_unique_circuits_mid = amp_pop20_circuits[selected_obj_idx_mid]

selected_obj_idx_low = np.where(
        amp_pop20_obj < 20
        )[0]
selected_unique_obj_low = amp_pop20_obj[selected_obj_idx_low]
selected_unique_circuits_low = amp_pop20_circuits[selected_obj_idx_low]
print(selected_unique_obj_low)

#### mid-range ON_rel

In [ ]:
amp_problem_200 = Amplifier(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=False, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200, num_processes=8, max_time=42, single_cell_tracking=False)

all_ON_rel_200 = []
for i, circuit in enumerate(selected_unique_circuits_mid):
    print("20 cell: ", selected_unique_obj_mid[i])
    ON_rel_200 = amp_problem_200.func(circuit)
    print("200 cell: ", ON_rel_200)
    all_ON_rel_200.append(ON_rel_200)
    print("topology", i, " done")
    print("")


amp_pop_comparison = {"20_cell": selected_unique_obj_mid, "200_cell": all_ON_rel_200}
amp_pop_df = pd.DataFrame(amp_pop_comparison)
amp_pop_df_pos = amp_pop_df.abs()
path_save = amp_pop_path + "pop200_comparison_mid-range/"
amp_pop_df_pos.to_csv(path_save+"pop_model_comparisons.csv")


In [ ]:
x_vals = [0]*len(amp_pop_df_pos)
jittered_x = x_vals + np.random.rand(
    len(x_vals))
amp_pop_df_pos["jittered_x"] = jittered_x

plt.figure(figsize=(3,3))
plt.plot(amp_pop_df_pos["jittered_x"], amp_pop_df_pos["20_cell"], label="20 cell model", linestyle="none", marker="o", markersize="4", zorder=1)
plt.plot(amp_pop_df_pos["jittered_x"], amp_pop_df_pos["200_cell"], label="200 cell model", linestyle="none", marker="o", markersize="4", zorder=2)
plt.xticks([])
plt.legend()
# plt.show()
plt.savefig(path_save+"20vs200_cell_ON_rel_mid.svg")

#### low ON_rel

In [ ]:
amp_problem_200 = Amplifier(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True, 
                        DsRed_inhibitor=False, num_dict={}, n_gen=None, probability_crossover=None, 
                        probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200, num_processes=8, max_time=42, single_cell_tracking=False)

all_ON_rel_200 = []
for i, circuit in enumerate(selected_unique_circuits_low):
    print("20 cell: ", selected_unique_obj_low[i])
    ON_rel_200 = amp_problem_200.func(circuit)
    print("200 cell: ", ON_rel_200)
    all_ON_rel_200.append(ON_rel_200)
    print("topology", i, " done")
    print("")


amp_pop_comparison = {"20_cell": selected_unique_obj_low, "200_cell": all_ON_rel_200}
amp_pop_df = pd.DataFrame(amp_pop_comparison)
amp_pop_df_pos = amp_pop_df.abs()
path_save = amp_pop_path + "pop200_comparison_low/"
amp_pop_df_pos.to_csv(path_save+"pop_model_comparisons.csv")

In [ ]:
x_vals = [0]*len(amp_pop_df_pos)
jittered_x = x_vals + np.random.rand(
    len(x_vals))
amp_pop_df_pos["jittered_x"] = jittered_x

plt.figure(figsize=(3,3))
plt.plot(amp_pop_df_pos["jittered_x"], amp_pop_df_pos["20_cell"], label="20 cell model", linestyle="none", marker="o", markersize="4", zorder=1)
plt.plot(amp_pop_df_pos["jittered_x"], amp_pop_df_pos["200_cell"], label="200 cell model", linestyle="none", marker="o", markersize="4", zorder=2)
plt.xticks([])
plt.ylabel("ON_rel")
plt.legend()
# plt.show()
plt.savefig(path_save+"20vs200_cell_ON_rel_low.svg")

### Signal conditioner: original hyperparameters

In [33]:
sc_pop_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_pop/synTF-R-DsR/Original_hyperparams/SC_original_hp_seed_0/"
obj_fname = "unique_objectives_df.pkl"
circuit_fname = "unique_circuits.pkl"

unique_obj_df = pd.read_pickle(sc_pop_path+obj_fname).abs()
with open(sc_pop_path+circuit_fname, "rb") as fid:
    sc_pop20_circuits = pickle.load(fid)
sc_pop20_circuits = sc_pop20_circuits.flatten()
# display(unique_obj_df)

selected_obj_mid = unique_obj_df[
    (unique_obj_df["ON_rel"] < 15) &
    ((unique_obj_df["FI_rel"] > 0.7) & (unique_obj_df["FI_rel"] < 0.8))
]
unique_obj_mid = selected_obj_mid.copy()
unique_ON_rel_mid = unique_obj_mid["ON_rel"].tolist()
unique_FI_rel_mid = unique_obj_mid["FI_rel"].tolist()

selected_obj_idx_m = selected_obj_mid.index.tolist()
selected_circuits_mid = sc_pop20_circuits[selected_obj_idx_m]

selected_obj_low = unique_obj_df[
    ((unique_obj_df["ON_rel"] > 15) & (unique_obj_df["ON_rel"] < 30)) &
    ((unique_obj_df["FI_rel"] > 0.4) & (unique_obj_df["FI_rel"] < 0.5))
]

unique_obj_low = selected_obj_low.copy()
unique_ON_rel_low = unique_obj_low["ON_rel"].tolist()
unique_FI_rel_low = unique_obj_low["FI_rel"].tolist()

selected_obj_idx_l = selected_obj_low.index.tolist()
selected_circuits_low = sc_pop20_circuits[selected_obj_idx_l]

# plt.plot(unique_obj_df["ON_rel"], unique_obj_df["FI_rel"],  linestyle="none", marker="o")
# plt.plot(unique_obj_mid["ON_rel"], unique_obj_mid["FI_rel"], linestyle="none", marker="o")
# plt.plot(unique_obj_low["ON_rel"], unique_obj_low["FI_rel"], linestyle="none", marker="o")


#### signal conditioner original hyperparameters mid-range objs

In [ ]:
signal_cond_200 = SignalConditioner(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True,
                                    DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None,
                                    probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200,
                                    num_processes=8, max_time=42, single_cell_tracking=False)

all_ON_rel_200 = []
all_FI_rel_200 = []
for i, circuit in enumerate(selected_circuits_mid):
    print("20 cell ON_rel, FI_rel: ", unique_ON_rel_mid[i], unique_FI_rel_mid[i])
    [ON_rel_200, FI_rel_200] = signal_cond_200.func(circuit)
    print("200 cell ON_rel, FI_rel: ", -ON_rel_200, -FI_rel_200)
    all_ON_rel_200.append(ON_rel_200)
    all_FI_rel_200.append(FI_rel_200)
    print("topology", i, " done")
    print("")


sc_pop_comparison = {"20_cell ON_rel": unique_ON_rel_mid,
                     "20_cell FI_rel": unique_FI_rel_mid,
                     "200_cell ON_rel": all_ON_rel_200,
                     "200_cell FI_rel": all_FI_rel_200}


sc_pop_df = pd.DataFrame(sc_pop_comparison)
sc_pop_df_pos = sc_pop_df.abs()
path_save = sc_pop_path + "pop200_comparison_mid/"
sc_pop_df_pos.to_csv(path_save+"pop_model_comparisons.csv")

In [ ]:
plt.figure(figsize=(3,3))
plt.plot(sc_pop_df_pos["20_cell ON_rel"], sc_pop_df_pos["20_cell FI_rel"], label="20 cell model", linestyle="none", marker="o", markersize="4", zorder=1)
plt.plot(sc_pop_df_pos["200_cell ON_rel"], sc_pop_df_pos["200_cell FI_rel"], label="200 cell model", linestyle="none", marker="o", markersize="4", zorder=2)
plt.xlabel("ON_rel")
plt.ylabel("FI_rel")
plt.legend()
# plt.show()
plt.savefig(path_save+"20vs200_cell_objs_mid.svg")

#### signal conditioner original hyperparameters low objs (FI_rel)

In [ ]:
signal_cond_200 = SignalConditioner(promo_node="P1", dose_specs=[5, 75, 5], max_part=2, inhibitor=True,
                                    DsRed_inhibitor=True, num_dict={}, n_gen=None, probability_crossover=None,
                                    probability_mutation=None, mutate_dose=True, pop=True, CI=None, Z_mat = Z_200,
                                    num_processes=8, max_time=42, single_cell_tracking=False)

all_ON_rel_200 = []
all_FI_rel_200 = []
for i, circuit in enumerate(selected_circuits_low):
    print("20 cell ON_rel, FI_rel: ", unique_ON_rel_low[i], unique_FI_rel_low[i])
    [ON_rel_200, FI_rel_200] = signal_cond_200.func(circuit)
    print("200 cell ON_rel, FI_rel: ", -ON_rel_200, -FI_rel_200)
    all_ON_rel_200.append(ON_rel_200)
    all_FI_rel_200.append(FI_rel_200)
    print("topology", i, " done")
    print("")


sc_pop_comparison = {"20_cell ON_rel": unique_ON_rel_low,
                     "20_cell FI_rel": unique_FI_rel_low,
                     "200_cell ON_rel": all_ON_rel_200,
                     "200_cell FI_rel": all_FI_rel_200}


sc_pop_df = pd.DataFrame(sc_pop_comparison)
sc_pop_df_pos = sc_pop_df.abs()
path_save = sc_pop_path + "pop200_comparison_low/"
sc_pop_df_pos.to_csv(path_save+"pop_model_comparisons.csv")

In [ ]:
plt.figure(figsize=(3,3))
plt.plot(sc_pop_df_pos["20_cell ON_rel"], sc_pop_df_pos["20_cell FI_rel"], label="20 cell model", linestyle="none", marker="o", markersize="4", zorder=1)
plt.plot(sc_pop_df_pos["200_cell ON_rel"], sc_pop_df_pos["200_cell FI_rel"], label="200 cell model", linestyle="none", marker="o", markersize="4", zorder=2)
plt.xlabel("ON_rel")
plt.ylabel("FI_rel")
plt.legend()
# plt.show()
# plt.savefig(path_save+"20vs200_cell_objs_low.svg")

## CI analysis 

### Use in_dict to compare two circuits

In [58]:
def sort_in_dict(in_dict):
    sorted_in_dict = {}
    for key, val in in_dict.items():
    # print(key, val)
        val_sorted = {}
        for subkey, subval in val.items():
            subval.sort()
        #     print(subkey, subval)
            val_sorted.update({subkey: subval})
        sorted_in_dict.update({key: val_sorted})
    return sorted_in_dict


def generalize_in_dict(in_dict):
    in_dict_general = {}
    # format of keys: "part|regulator1|regulator2|..."
    for key, value in in_dict.items():
        i = 0
        key_reg = key[0]
        if len(value["P"]) > 0:
            key_reg += "P"
        if len(value["Z"]) > 0:
            key_regZ = ""
            for Zreg in value["Z"]:
                if Zreg == key:
                    key_regZ += "A"
                else:
                    key_regZ += "Z"
            key_regZ = "".join(sorted(key_regZ))
            key_reg += key_regZ
        if len(value["I"]) > 0:
            key_regI = ""
            for Ireg in value["I"]:
                if Ireg == key:
                    key_regI += "A"
                else:
                    key_regI += "I"
            key_regI = "".join(sorted(key_regI))
            key_reg += key_regI
        in_dict_general.update({key_reg: value})
    
    return in_dict_general

### CI topologies

In [ ]:
path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Amplifier_pop/Original_hyperparams_vary_dose/Amplifier_pop_vary_dose_original_hp_seed_0/"
CI_fname = "2024-08-23_results_analysis_full_CI/selected_results_ON_rel_full_CI.pkl"
unique_objs_fname = ""
amp_CI = pd.read_pickle(path+CI_fname)

objs0 = []
in_dict0 = {'Z6': {'P': [], 'Z': ['Z6', 'Z2'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z6', 'Z2'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z6', 'Z2'], 'I': []}
}
sorted_in_dict0 = sort_in_dict(in_dict0)

objs1 = []
in_dict1 = {'Z6': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z2'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []}
}
sorted_in_dict1 = sort_in_dict(in_dict1)

objs2 = []
in_dict2 = {'Z6': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z2', 'Z6'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z6'], 'I': []}
}
sorted_in_dict2 = sort_in_dict(in_dict2)

objs3 = []
in_dict3 = {'Z6': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z6'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []}
}
sorted_in_dict3 = sort_in_dict(in_dict3)

objs4 = []
in_dict4 = {'Z6': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z6'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z6'], 'I': []}
}
sorted_in_dict4 = sort_in_dict(in_dict4)

objs5 = []
in_dict5 = {'Z6': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z2'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z6'], 'I': []}
}
sorted_in_dict5 = sort_in_dict(in_dict5)


in_dict0_general = generalize_in_dict(in_dict0)
# print(in_dict0_general.keys())
in_dict1_general = generalize_in_dict(in_dict1)
# print(in_dict1_general.keys())
in_dict2_general = generalize_in_dict(in_dict2)
# print(in_dict2_general.keys())
in_dict3_general = generalize_in_dict(in_dict3)
# print(in_dict3_general.keys())
in_dict4_general = generalize_in_dict(in_dict4)
# print(in_dict4_general.keys())
in_dict5_general = generalize_in_dict(in_dict5)
# print(in_dict5_general.keys())
dict_list = [sorted_in_dict0, sorted_in_dict1, sorted_in_dict2, sorted_in_dict3, sorted_in_dict4, sorted_in_dict5]
dict_list_general = [in_dict0_general, in_dict1_general, in_dict2_general, in_dict3_general, in_dict4_general, in_dict5_general]
keys_list = [d.keys() for d in dict_list_general]


opt_circuits = {
    "opt_0_topology": [],
    "opt_0_objs": [],
    "opt_0_doses": [],
    "opt_1_topology": [],
    "opt_1_objs": [],
    "opt_1_doses": [],
    "opt_2_topology": [],
    "opt_2_objs": [],
    "opt_2_doses": [],
    "opt_3_topology": [],
    "opt_3_objs": [],
    "opt_3_doses": [],
    "opt_4_topology": [],
    "opt_4_objs": [],
    "opt_4_doses": [],
    "opt_5_topology": [],
    "opt_5_objs": [],
    "opt_5_doses": [],
}
opt_variants = {
    "opt_0_var_topology": [],
    "opt_0_var_objs": [],
    "opt_0_var_doses": [],
    "opt_1_var_topology": [],
    "opt_1_var_objs": [],
    "opt_1_var_doses": [],
    "opt_2_var_topology": [],
    "opt_2_var_objs": [],
    "opt_2_var_doses": [],
    "opt_3_var_topology": [],
    "opt_3_var_objs": [],
    "opt_3_var_doses": [],
    "opt_4_var_topology": [],
    "opt_4_var_objs": [],
    "opt_4_var_doses": [],
    "opt_5_var_topology": [],
    "opt_5_var_objs": [],
    "opt_5_var_doses": [],
}

for index, row in amp_CI.iterrows():
    # print("CI circuit ", index)
    row_in_dict = row["Parts order"]
    sorted_in_dict = sort_in_dict(row_in_dict)
    row_in_dict_general = generalize_in_dict(row_in_dict)
    if sorted_in_dict in dict_list:
        opt_circuit_idx = keys_list.index(row_in_dict_general.keys())
        # print("circuit is opt ", opt_circuit_idx)
        opt_circuits["opt_"+str(opt_circuit_idx)+"_topology"].append(row["Topology"])
        opt_circuits["opt_"+str(opt_circuit_idx)+"_objs"].append(row["ON_rel"])
        opt_circuits["opt_"+str(opt_circuit_idx)+"_doses"].append(row["Topology"].dose)
    elif row_in_dict_general.keys() in keys_list:
        variant_idx = keys_list.index(row_in_dict_general.keys())
        # print("circuit is variant of opt ", keys_list.index(row_in_dict_general.keys()))
        opt_variants["opt_"+str(variant_idx)+"_var_topology"].append(row["Topology"])
        opt_variants["opt_"+str(variant_idx)+"_var_objs"].append(row["ON_rel"])
        opt_variants["opt_"+str(variant_idx)+"_var_doses"].append(row["Topology"].dose)


    # print("*****")
opt_circuits_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in opt_circuits.items()]))
opt_variants_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in opt_variants.items()]))
# opt_circuits_df_nonan = opt_circuits_df.fillna(value="", inplace=False)
opt_objs = opt_circuits_df.drop(list(
    opt_circuits_df.filter(regex="topology")
    ), axis=1)
opt_objs = opt_objs.drop(list(
    opt_circuits_df.filter(regex="doses")
    ), axis=1)
# display(opt_objs)
# display(opt_circuits_df_nonan)
opt_variant_objs = opt_variants_df.drop(list(
    opt_variants_df.filter(regex="topology")
    ), axis=1)
opt_variant_objs = opt_variant_objs.drop(list(
    opt_variants_df.filter(regex="doses")
    ), axis=1)

# opt_circuits_df.to_pickle(path+"topologies_in_CI.pkl")
# opt_circuits_df.to_csv(path+"topologies_in_CI.csv")
# opt_variants_df.to_pickle(path+"topology_variants_in_CI.pkl")
# opt_variants_df.to_csv(path+"topology_variants_in_CI.csv")

opt_objs_T = opt_objs.transpose()
opt_objs_T["opt_topo"] = opt_objs_T.index
# display(opt_objs_T)
opt_variant_objs_T = opt_variant_objs.transpose()
opt_variant_objs_T["opt_topo"] = opt_variant_objs_T.index

opt_objs_plot = pd.melt(frame=opt_objs_T,
                        id_vars="opt_topo",
                        var_name="column_name",
                        value_name="ON_rel")
opt_objs_plot.dropna(inplace=True)

opt_variant_objs_plot = pd.melt(frame=opt_variant_objs_T,
                        id_vars="opt_topo",
                        var_name="column_name",
                        value_name="ON_rel")
opt_variant_objs_plot.dropna(inplace=True)

x_vals = [0]*len(opt_objs_plot)
print(len(x_vals))
np.random.seed(0)
jittered_x = x_vals + np.random.rand(
    len(x_vals))
opt_objs_plot["jittered_x"] = jittered_x
# display(opt_objs_plot)

x_vals_var = [0]*len(opt_variant_objs_plot)
jittered_x_var = x_vals_var + np.random.rand(
    len(x_vals_var))
opt_variant_objs_plot["jittered_x"] = jittered_x_var
# fig, ax = plt.subplots(1, 1, figsize=(3, 3))
# sns.scatterplot(data= opt_variant_objs_plot, x="jittered_x", y="ON_rel", ax=ax, hue="opt_topo", palette=colors)

fig, ax = plt.subplots(1, 1, figsize=(1.9, 1.75))
sns.scatterplot(data= opt_objs_plot, x="jittered_x", y="ON_rel", ax=ax, hue="opt_topo", palette=colors, s=20, legend=False)
sns.scatterplot(data= opt_variant_objs_plot, x="jittered_x", y="ON_rel", ax=ax, hue="opt_topo", palette=[colors[2], colors[5]], marker= "X", s=30, legend=False)
lower_bound = [63.11786016]*len(opt_objs_plot)
upper_bound = [63.50911412131601]*len(opt_objs_plot)
jittered_x.sort()
ax.fill_between(jittered_x, lower_bound, upper_bound, alpha=0.3, color="grey", zorder=3)
ax.set_xticks([])
ax.set_yticks([63.1, 63.2, 63.3, 63.4, 63.5])
ax.set_xlabel("")
# plt.show()
plt.savefig(path+"topologies_in_CI.svg")

### Sub-opt topologies

In [ ]:
path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Amplifier_pop/Original_hyperparams_vary_dose/Amplifier_pop_vary_dose_original_hp_seed_0/"
topologies_fname = "unique_circuits.pkl"
objs_fname = "unique_objectives.pkl"

with open(path+topologies_fname, "rb") as fid:
    unique_topos = pickle.load(fid)
unique_topos = unique_topos.flatten()
with open(path+objs_fname, "rb") as fid:
    unique_objs = pickle.load(fid)
unique_objs = abs(unique_objs.flatten())
unique_results = np.column_stack((unique_topos, unique_objs))
# print(unique_results)

unique_results_df = pd.DataFrame(data=unique_results, columns=["Topology", "ON_rel"])
unique_results_subopt = unique_results_df[unique_results_df["ON_rel"] < 63.11786016]
display(unique_results_subopt)

In [ ]:
objs0 = []
in_dict0 = {'Z6': {'P': [], 'Z': ['Z6', 'Z2'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z6', 'Z2'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z6', 'Z2'], 'I': []}
}
topo0_edges = [('P1', 'Z2'), ('Z6', 'Z6'), ('Z6', 'Rep'), ('Z6', 'Z2'), ('Z2', 'Z2'), ('Z2', 'Z6'), ('Z2', 'Rep')]
topo0 = nx.DiGraph()
topo0.add_edges_from(topo0_edges)
shortest_path0 = nx.shortest_path(topo0, "P1", "Rep")
# print(shortest_path0)
sorted_in_dict0 = sort_in_dict(in_dict0)

objs1 = []
in_dict1 = {'Z6': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z2'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []}
}
topo1_edges = [('P1', 'Z2'), ('Z6', 'Z6'), ('Z6', 'Rep'), ('Z2', 'Z2'), ('Z2', 'Z6'), ('Z2', 'Rep')]
topo1 = nx.DiGraph()
topo1.add_edges_from(topo1_edges)
shortest_path1 = nx.shortest_path(topo1, "P1", "Rep")
# print(shortest_path1)
sorted_in_dict1 = sort_in_dict(in_dict1)

objs2 = []
in_dict2 = {'Z6': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z2', 'Z6'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z6'], 'I': []}
}
topo2_edges = [('P1', 'Z2'), ('Z6', 'Z6'), ('Z6', 'Rep'), ('Z6', 'Z2'), ('Z2', 'Z2'), ('Z2', 'Z6')]
topo2 = nx.DiGraph()
topo2.add_edges_from(topo2_edges)
shortest_path2 = nx.shortest_path(topo2, "P1", "Rep")
# print(shortest_path2)
sorted_in_dict2 = sort_in_dict(in_dict2)

objs2_Z7 = []
in_dict2_Z7 = {'Z6': {'P': [], 'Z': ['Z7', 'Z6'], 'I': []},
            'Z7': {'P': ['P1'], 'Z': ['Z7', 'Z6'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z6'], 'I': []}
}
topo2_Z7_edges = [('P1', 'Z7'), ('Z6', 'Z6'), ('Z6', 'Rep'), ('Z6', 'Z7'), ('Z7', 'Z7'), ('Z7', 'Z6')]
topo2_Z7 = nx.DiGraph()
topo2_Z7.add_edges_from(topo2_Z7_edges)
shortest_path2_Z7 = nx.shortest_path(topo2_Z7, "P1", "Rep")
sorted_in_dict2 = sort_in_dict(in_dict2)
sorted_in_dict2_Z7 = sort_in_dict(in_dict2_Z7)

objs3 = []
in_dict3 = {'Z6': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z6'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []}
}
topo3_edges = [('P1', 'Z2'), ('Z6', 'Z6'), ('Z6', 'Rep'), ('Z6', 'Z2'), ('Z2', 'Z6'), ('Z2', 'Rep')]
topo3 = nx.DiGraph()
topo3.add_edges_from(topo3_edges)
shortest_path3 = nx.shortest_path(topo3, "P1", "Rep")
sorted_in_dict3 = sort_in_dict(in_dict3)

objs4 = []
in_dict4 = {'Z6': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z6'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z6'], 'I': []}
}
topo4_edges = [('P1', 'Z2'), ('Z6', 'Z6'), ('Z6', 'Rep'), ('Z6', 'Z2'), ('Z2', 'Z6')]
topo4 = nx.DiGraph()
topo4.add_edges_from(topo4_edges)
shortest_path4 = nx.shortest_path(topo4, "P1", "Rep")
sorted_in_dict4 = sort_in_dict(in_dict4)

objs5 = []
in_dict5 = {'Z6': {'P': [], 'Z': ['Z2', 'Z6'], 'I': []},
            'Z2': {'P': ['P1'], 'Z': ['Z2'], 'I': []},
            'Rep': {'P': [], 'Z': ['Z6'], 'I': []}
}
topo5_edges = [('P1', 'Z2'), ('Z6', 'Z6'), ('Z6', 'Rep'), ('Z2', 'Z2'), ('Z2', 'Z6')]
topo5 = nx.DiGraph()
topo5.add_edges_from(topo5_edges)
shortest_path5 = nx.shortest_path(topo5, "P1", "Rep")
sorted_in_dict5 = sort_in_dict(in_dict5)


in_dict0_general = generalize_in_dict(in_dict0)
# print(in_dict0_general.keys())
in_dict1_general = generalize_in_dict(in_dict1)
# print(in_dict1_general.keys())
in_dict2_general = generalize_in_dict(in_dict2)
# print(in_dict2_general.keys())
in_dict2_Z7_general = generalize_in_dict(in_dict2_Z7)
# print(in_dict2_general.keys())
in_dict3_general = generalize_in_dict(in_dict3)
# print(in_dict3_general.keys())
in_dict4_general = generalize_in_dict(in_dict4)
# print(in_dict4_general.keys())
in_dict5_general = generalize_in_dict(in_dict5)
# print(in_dict5_general.keys())
dict_list = [sorted_in_dict0, sorted_in_dict1, sorted_in_dict2, sorted_in_dict3, sorted_in_dict4, sorted_in_dict5]
dict_list_general = [in_dict0_general, in_dict1_general, in_dict2_general, in_dict3_general, in_dict4_general, in_dict5_general]
keys_list = [d.keys() for d in dict_list_general]
shortest_path_list = [shortest_path0, shortest_path1, shortest_path2, shortest_path3, shortest_path4, shortest_path5]


opt_circuits = {
    "opt_0_topology": [],
    "opt_0_objs": [],
    "opt_0_doses": [],
    "opt_1_topology": [],
    "opt_1_objs": [],
    "opt_1_doses": [],
    "opt_2_topology": [],
    "opt_2_objs": [],
    "opt_2_doses": [],
    "opt_2_Z7_topology": [],
    "opt_2_Z7_objs": [],
    "opt_2_Z7_doses": [],
    "opt_3_topology": [],
    "opt_3_objs": [],
    "opt_3_doses": [],
    "opt_4_topology": [],
    "opt_4_objs": [],
    "opt_4_doses": [],
    "opt_5_topology": [],
    "opt_5_objs": [],
    "opt_5_doses": [],
}
opt_variants = {
    "opt_0_var_topology": [],
    "opt_0_var_objs": [],
    "opt_0_var_doses": [],
    "opt_1_var_topology": [],
    "opt_1_var_objs": [],
    "opt_1_var_doses": [],
    "opt_2_var_topology": [],
    "opt_2_var_objs": [],
    "opt_2_var_doses": [],
    "opt_3_var_topology": [],
    "opt_3_var_objs": [],
    "opt_3_var_doses": [],
    "opt_4_var_topology": [],
    "opt_4_var_objs": [],
    "opt_4_var_doses": [],
    "opt_5_var_topology": [],
    "opt_5_var_objs": [],
    "opt_5_var_doses": [],
}

for index, row in unique_results_subopt.iterrows():
    # print("CI circuit ", index)
    row_in_dict = row["Topology"].in_dict
    sorted_in_dict = sort_in_dict(row_in_dict)
    row_in_dict_general = generalize_in_dict(row_in_dict)
    if sorted_in_dict in dict_list:
        opt_circuit_idx = keys_list.index(row_in_dict_general.keys())
        # print("circuit is opt ", opt_circuit_idx)
        opt_circuits["opt_"+str(opt_circuit_idx)+"_topology"].append(row["Topology"])
        opt_circuits["opt_"+str(opt_circuit_idx)+"_objs"].append(row["ON_rel"])
        opt_circuits["opt_"+str(opt_circuit_idx)+"_doses"].append(row["Topology"].dose)
    elif sorted_in_dict == sorted_in_dict2_Z7:
        opt_circuits["opt_2_Z7_topology"].append(row["Topology"])
        opt_circuits["opt_2_Z7_objs"].append(row["ON_rel"])
        opt_circuits["opt_2_Z7_doses"].append(row["Topology"].dose)
    elif row_in_dict_general.keys() in keys_list:
        possible_variant = keys_list.index(row_in_dict_general.keys())
        topo_graph = row["Topology"].graph
        shortest_path = nx.shortest_path(topo_graph, "P1", "Rep")
        if len(shortest_path) == len(shortest_path_list[possible_variant]):
        #     print(row["Topology"].in_dict)
        #     print("circuit is variant of opt ", keys_list.index(row_in_dict_general.keys()))
        #     print("***")
            opt_variants["opt_"+str(possible_variant)+"_var_topology"].append(row["Topology"])
            opt_variants["opt_"+str(possible_variant)+"_var_objs"].append(row["ON_rel"])
            opt_variants["opt_"+str(possible_variant)+"_var_doses"].append(row["Topology"].dose)

    # print("*****")
opt_circuits_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in opt_circuits.items()]))
opt_variants_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in opt_variants.items()]))
opt_circuits_df_nonan = opt_circuits_df.fillna(value="", inplace=False)
# opt_variants_df_nonan = opt_variants_df.fillna(value="", inplace=False)
opt_objs = opt_circuits_df.drop(list(
    opt_circuits_df.filter(regex="topology")
    ), axis=1)
opt_objs = opt_objs.drop(list(
    opt_circuits_df.filter(regex="doses")
    ), axis=1)
opt_variant_objs = opt_variants_df.drop(list(
    opt_variants_df.filter(regex="topology")
    ), axis=1)
opt_variant_objs = opt_variant_objs.drop(list(
    opt_variants_df.filter(regex="doses")
    ), axis=1)

# display(opt_objs)
# display(opt_circuits_df_nonan)

# opt_circuits_df.to_pickle(path+"topologies_sub_opt.pkl")
# opt_circuits_df.to_csv(path+"topologies_sub_opt.csv")
# opt_variants_df.to_pickle(path+"topology_variants_sub_opt.pkl")
# opt_variants_df.to_csv(path+"topology_variants_sub_opt.csv")

opt_objs_T = opt_objs.transpose()
opt_objs_T["opt_topo"] = opt_objs_T.index
opt_variant_objs_T = opt_variant_objs.transpose()
opt_variant_objs_T["opt_topo"] = opt_variant_objs_T.index
# display(opt_objs_T)

opt_objs_plot = pd.melt(frame=opt_objs_T,
                        id_vars="opt_topo",
                        var_name="column_name",
                        value_name="ON_rel")
opt_objs_plot.dropna(inplace=True)
# print(len(opt_objs_plot))
opt_variant_objs_plot = pd.melt(frame=opt_variant_objs_T,
                        id_vars="opt_topo",
                        var_name="column_name",
                        value_name="ON_rel")
opt_variant_objs_plot.dropna(inplace=True)
# print(len(opt_variant_objs_plot))
x_vals = [0]*len(opt_objs_plot)
jittered_x = x_vals + np.random.rand(
    len(x_vals))
opt_objs_plot["jittered_x"] = jittered_x
display(opt_objs_plot)
fig, ax = plt.subplots(1, 1, figsize=(3, 3))
sns.scatterplot(data= opt_objs_plot, x="jittered_x", y="ON_rel", ax=ax, hue="opt_topo", palette=colors)
plt.ylim([0, 65])
plt.show()

# x_vals_var = [0]*len(opt_variant_objs_plot)
# jittered_x_var = x_vals_var + np.random.rand(
#     len(x_vals_var))
# opt_variant_objs_plot["jittered_x"] = jittered_x_var
# fig, ax = plt.subplots(1, 1, figsize=(3, 3))
# sns.scatterplot(data= opt_variant_objs_plot, x="jittered_x", y="ON_rel", ax=ax, hue="opt_topo", palette=colors)
# plt.ylim([0, 65])
# plt.show()
# opt_variant_objs_low_plot = opt_variant_objs_plot[opt_variant_objs_plot["ON_rel"] < 60]
# print(len(opt_variant_objs_low_plot))


# opt_objs_low_plot = opt_objs_plot[opt_objs_plot["ON_rel"] < 60]
# # display(opt_objs_low_plot)
# print(len(opt_objs_low_plot))

# x_vals = [0]*len(opt_objs_low_plot)
# jittered_x = x_vals + np.random.rand(
#     len(x_vals))
# opt_objs_low_plot["jittered_x"] = jittered_x
# # # display(opt_objs_plot)
# fig, ax = plt.subplots(1, 1, figsize=(3, 3))
# sns.scatterplot(data= opt_objs_low_plot, x="jittered_x", y="ON_rel", ax=ax, hue="opt_topo", palette=colors)
# plt.ylim([0, 65])
# plt.show()

### Find objectives that fall within pareto CI

In [ ]:
# sc_pop_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_pop/synTF-R-DsR/Original_hyperparams/SC_original_hp_seed_0/"
sc_pop_path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_pop/synTF-R-DsR/Single_cell_model_opt_hyperparams/run2_ngen80/Single_cell_opt_hp_seed_0/"

obj_fname = "unique_objectives_df.pkl"
pareto_fname = "final_objectives_df.pkl"

circuit_fname = "unique_circuits.pkl"
# CI_path = "2024-09-06_Z_matrix_sampling/Z_matrix_sampling_for_CI.pkl"

pareto_front_objectives = pd.read_pickle(
    sc_pop_path+pareto_fname
)
selected_objectives = (
    pareto_front_objectives.drop_duplicates()
)
display(selected_objectives)
unique_obj_df = pd.read_pickle(sc_pop_path+obj_fname).abs()
with open(sc_pop_path+circuit_fname, "rb") as fid:
    sc_pop20_circuits = pickle.load(fid)
sc_pop20_circuits = sc_pop20_circuits.flatten()

# CI_sampling = pd.read_pickle(sc_pop_path+CI_path)
# display(CI_sampling)
# print(max(CI_sampling["ON_rel_std_error"]), max(CI_sampling["FI_rel_std_error"]))


In [ ]:
obj_labels = ["ON_rel", "FI_rel"]
CI_metric_maxes = [0.7285110622796277, 0.026919138877793805]
obj1_CI = CI_metric_maxes[0]
obj2_CI = CI_metric_maxes[1]

upper_obj1 = np.abs(np.array(selected_objectives[obj_labels[0]]))
sorted_upper_idx = np.argsort(upper_obj1)
sorted_upper_obj1 = upper_obj1[sorted_upper_idx]

lower_obj1 = np.array([i-obj1_CI for i in upper_obj1])
sorted_lower_idx = np.argsort(lower_obj1)
sorted_lower_obj1 = lower_obj1[sorted_lower_idx]

upper_obj2 = np.abs(np.array(selected_objectives[obj_labels[1]]))
sorted_upper_obj2 = upper_obj2[sorted_upper_idx]

lower_obj2 = np.array([i-obj2_CI for i in upper_obj2])
sorted_lower_obj2 = lower_obj2[sorted_lower_idx]

xfill = np.sort(np.concatenate([upper_obj1, lower_obj1]))

y1fill = np.interp(xfill, sorted_upper_obj1, sorted_upper_obj2)
y2fill = np.interp(xfill, sorted_lower_obj1, sorted_lower_obj2)

unique_obj_df_CI = unique_obj_df[
    (unique_obj_df["FI_rel"] >= np.interp(unique_obj_df["ON_rel"], xfill, y2fill)) &
    (unique_obj_df["FI_rel"] <= np.interp(unique_obj_df["ON_rel"], xfill, y1fill))
]
# display(unique_obj_df_CI)
CI_idx = unique_obj_df_CI.index.tolist()
unique_circuits_CI = sc_pop20_circuits[CI_idx]
unique_results_CI = unique_obj_df_CI.copy()
unique_results_CI["Topology"] = unique_circuits_CI
unique_results_CI["Edge_list"] = 0
unique_results_CI["Edge_list"] = unique_results_CI["Edge_list"].astype(object)
unique_results_CI["Edge_list"] = [topology.edge_list for topology in unique_circuits_CI]
high_FI_df = unique_results_CI.copy()
high_FI_df = high_FI_df[high_FI_df["FI_rel"] > 1]
display(high_FI_df)
for index, row in high_FI_df.iterrows():
    # print(row["Edge_list"])
    row["Topology"].plot_graph()


# plt.plot(unique_obj_df["ON_rel"], unique_obj_df["FI_rel"], linestyle="none", marker="o", markersize="2", color="grey")
# plt.plot(unique_obj_df_CI["ON_rel"], unique_obj_df_CI["FI_rel"], linestyle="none", marker="o", markersize="2")
# plt.plot(xfill, y1fill, linewidth="2")
# plt.plot(xfill, y2fill, linewidth="2")

In [ ]:
path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_single_cell/synTF-R-DsR/Optimized_hyperparams_fixed_pop_opt_stdev_ngen50/run2_ngen80/SC_opt_hp_ngen50_run2_seed_7/"
fname = "all_objectives.pkl"
fname_circuits = "all_circuits.pkl"
all_objs = pd.read_pickle(path+fname)
all_circuits = pd.read_pickle(path+fname_circuits)
unique_objs, unique_idx = np.unique(all_objs, axis=0, return_index=True)
unique_objs = abs(unique_objs)
# print(len(unique_objs))
unique_circuits = all_circuits[unique_idx]
# with open(path+"unique_circuits.pkl", "wb") as fid:
#     pickle.dump(unique_circuits, fid)
# unique_obj_df = pd.DataFrame(unique_objs, columns=["ON_rel", "FI_rel"])
# unique_obj_df.to_pickle(path+"unique_objectives.pkl")
# fig, ax = plt.subplots(1, 1, figsize=(2.25, 2.25))
sns.scatterplot(data=unique_obj_df, x="ON_rel", y="FI_rel", color="k", ax=ax, s=8, linewidth=0.0)
# plt.savefig(path+"unique_obj_scatter.svg")

In [25]:
unique_obj_high = unique_obj_df[unique_obj_df["FI_rel"] > 1]
idx_high = unique_obj_high.index.tolist()
unique_circuits_high = unique_circuits[idx_high]
# display(unique_obj_high)
edge_list_high = []
doses_high = []
for i, circuit in enumerate(unique_circuits_high):
    edge_list_high.append(circuit[0].edge_list)
    doses_high.append(circuit[0].dose)
    # circuit[0].plot_graph()
unique_results_high = unique_obj_high.copy()
unique_results_high["Edge_list"] = 0
unique_results_high["Edge_list"] = unique_results_high["Edge_list"].astype(object)
unique_results_high["Edge_list"] = edge_list_high
unique_results_high["Doses"] = 0
unique_results_high["Doses"] = unique_results_high["Doses"].astype(object)
unique_results_high["Doses"] = doses_high
# display(unique_results_high)
unique_results_high.to_csv(path+"FI_rel_gt1.csv")

# fig, ax = plt.subplots(1, 1, figsize=(2.25, 2.25))
# sns.scatterplot(data=unique_obj_high, x="ON_rel", y="FI_rel", color="k", ax=ax, s=8, linewidth=0.0)
# plt.show()

In [ ]:
CI_stderrs = [0.7285110622796277, 0.026919138877793805]

In [ ]:
import pickle
path1 = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_single_cell/synTF-R-DsR/Original_hyperparams/run4_ngen120/2024-10-04_Signal_cond_single_DsRED_original_hps_ngen120_seed_4/hypervolumes.pkl"
path2 = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_single_cell/synTF-R-DsR/Optimized_hyperparams_fixed_pop_max_hv_ngen50/run4_ngen120/2024-10-04_Signal_cond_single_DsRED_opt_hps_run4_seed_5/hypervolumes.pkl"
with open(path2, "rb") as fid:
    hvs = pickle.load(fid)

print(hvs[-1])

## Signal Conditioner single cell 1000+ generations hypervolume analysis

### Compare final population to combinatorial pareto front

In [ ]:
from pymoo.indicators.hv import HV
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
from rankcrowding import RankAndCrowding
ref_point = np.array([0, 0])
hv = HV(ref_point=ref_point)

path_sc_combo_pareto = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Combinatorial_search/sig_cond_pareto_dsred1.pkl"
path_sc_combo = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Combinatorial_search/"

with open(path_sc_combo_pareto, "rb") as fid:
    sc_combo_pareto = pickle.load(fid)
sc_combo_pareto = abs(sc_combo_pareto)
sc_pareto_low_FI_idx = np.where(sc_combo_pareto[:,1] <= 1.0)
sc_pareto_high_FI_idx = np.where(sc_combo_pareto[:,1] > 1.0)
sc_pareto_low_FI = sc_combo_pareto[sc_pareto_low_FI_idx]
sc_pareto_high_FI = sc_combo_pareto[sc_pareto_high_FI_idx]

ref_point2 = np.array([0, -1*max(sc_pareto_low_FI[:, 1])])
hv2 = HV(ref_point=ref_point2)

pareto_low_FI_hv = hv(sc_pareto_low_FI*-1)
pareto_high_FI_hv = hv2(sc_pareto_high_FI*-1)
print("combo low FI hv: ", pareto_low_FI_hv)
print("combo high FI hv: ", pareto_high_FI_hv)
print("combo total hv: ", hv(sc_combo_pareto*-1))


rows, cols = sc_combo_pareto.shape

# Defining a data type
dt = {'names':['f{}'.format(i) for i in range(cols)],
       'formats':cols * [sc_combo_pareto.dtype]}

sorted_combo_pareto = sc_combo_pareto[np.argsort(sc_combo_pareto[:, 1])]
sorted_combo_pareto = np.around(sorted_combo_pareto, decimals=9)

fig2 = plt.figure(figsize=(2.5, 2.5))
plt.plot(sc_combo_pareto[:,0], sc_combo_pareto[:,1], linestyle="none", marker="o", markersize="2.5", label="combinatorial")
plt.xlabel("ON_rel")
plt.ylabel("FI_rel")
plt.legend()
# fig2.savefig(path_sc_combo +  "/signal_cond_pareto.svg")

In [ ]:
path_5000_all = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_single_cell/synTF-R-DsR/Optimized_hyperparams_fixed_pop_max_hv_ngen50/run7_ngen5000/"
path_10000_all = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_single_cell/synTF-R-DsR/Optimized_hyperparams_fixed_pop_max_hv_ngen50/run8_ngen10000/"
path_5000 = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_single_cell/synTF-R-DsR/Optimized_hyperparams_fixed_pop_max_hv_ngen50/run7_ngen5000/2025-01-09_Signal_cond_single_DsRED_opt_hps0_ngen5000_seed_"
path_10000 = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_single_cell/synTF-R-DsR/Optimized_hyperparams_fixed_pop_max_hv_ngen50/run8_ngen10000/2025-01-14_Signal_cond_single_DsRED_opt_hps0_ngen10000_seed_"
fname_obj = "final_objectives_df.pkl"
paths = [path_10000]

for path in paths:
    ngens = path[-11:-6]
    # print(ngens)
    fig1, axs = plt.subplots(1, 2, figsize=(5.5, 2))
    xticks = []
    high_FI_hv_list = []
    low_FI_hv_list = []
    total_hv_list = []
    for seed in range(0, 10):
        print("seed: ", seed)
        final_objs_GA_path = path + str(seed) + "/" + fname_obj
        with open(final_objs_GA_path, "rb") as fid:
            final_objs_GA = pickle.load(fid)

        final_objs_GA = final_objs_GA.drop("type", axis=1)
        final_objs_arr = final_objs_GA.to_numpy()
        final_objs_arr = abs(final_objs_arr)

        low_FI_objs_idx = np.where(final_objs_arr[:,1] <= 1.0)
        high_FI_objs_idx = np.where(final_objs_arr[:,1] > 1.0)
        low_FI_objs_arr = final_objs_arr[low_FI_objs_idx]
        high_FI_objs_arr = final_objs_arr[high_FI_objs_idx]
        # print(low_FI_objs_arr)

        ref_point_2_GA = np.array([0, -1*max(low_FI_objs_arr[:, 1])])
        hv2_GA = HV(ref_point=ref_point_2_GA)

        hv_GA = hv(final_objs_arr*-1)
        low_FI_hv_GA = hv(low_FI_objs_arr*-1)
        high_FI_hv_GA = hv2_GA(high_FI_objs_arr*-1)
        high_FI_hv_list.append(high_FI_hv_GA)
        low_FI_hv_list.append(low_FI_hv_GA)
        total_hv_list.append(hv_GA)
        print("GA", ngens, "gens low FI hv:", low_FI_hv_GA)
        print("GA", ngens, "gens high FI hv:", high_FI_hv_GA)
        
        print("GA", ngens, "gens total hv:", hv_GA)

        round_objs_GA = np.around(final_objs_arr, decimals=9)
        sorted_objs_GA = round_objs_GA[np.argsort(round_objs_GA[:, 1])]

        not_in_GA = np.setdiff1d(sorted_combo_pareto.copy().view(dt), sorted_objs_GA.copy().view(dt))
        not_in_GA = not_in_GA.view(sorted_combo_pareto.dtype).reshape(-1, cols)
        high_FI_idx = np.where(not_in_GA[:,1] > 1.0)
        high_FI_not_in_GA = (not_in_GA[high_FI_idx])*-1
        low_FI_idx = np.where(not_in_GA[:,1] < 1.0)
        low_FI_not_in_GA = (not_in_GA[low_FI_idx])*-1
        print("number of objectives with FI_rel > 1 that are not in GA final pop:", len(high_FI_not_in_GA))
        print("number of objectives with FI_rel < 1 that are not in GA final pop:", len(low_FI_not_in_GA))

        GA_pareto_intersect = np.intersect1d(sorted_objs_GA.copy().view(dt), sorted_combo_pareto.copy().view(dt))
        GA_pareto_intersect = GA_pareto_intersect.view(sorted_combo_pareto.dtype).reshape(-1, cols)
        print("number of objectives in GA final pop that are on combo pareto:", len(GA_pareto_intersect))

        fig2 = plt.figure(figsize=(2.5, 2.5))
        plt.plot(sc_combo_pareto[:,0], sc_combo_pareto[:,1], linestyle="none", marker="o", markersize="2.5", label="combinatorial")
        plt.plot(sorted_objs_GA[:,0], sorted_objs_GA[:,1], linestyle="none", marker="o", markersize="2.75", fillstyle='none', label="GA")
        plt.xlabel("ON_rel")
        plt.ylabel("FI_rel")
        plt.legend()
        # fig2.savefig(path + str(seed) + "/final_objectives_vs_combo.svg")
        # plt.close(fig2)
        # plt.show()

        axs[0].bar(seed+1, low_FI_hv_GA, width=0.5, align="center", color="grey")
        axs[1].bar(seed+1, high_FI_hv_GA, width=0.5, align="center", color="grey")
        xticks.append(seed+1)
    axs[0].axhline(y=pareto_low_FI_hv, linestyle="dashed", color="k", label="combinatorial")
    axs[0].set_xticks(xticks)
    axs[0].set_xticklabels([str(seed) for seed in range(10)])
    axs[0].set_xlabel("GA seed")
    axs[0].set_ylim([43.5,45])
    axs[0].set_ylabel("hypervolume")
    axs[0].set_title("FI_rel < 1 hypervolume")
    axs[0].legend(loc="upper right")

    axs[1].axhline(y=pareto_high_FI_hv, linestyle="dashed", color="k", label="combinatorial")
    axs[1].set_xticks(xticks)
    axs[1].set_xticklabels([str(seed) for seed in range(10)])
    axs[1].set_xlabel("GA seed")
    axs[1].set_ylim([0,1.5])
    axs[1].set_ylabel("hypervolume")
    axs[1].set_title("FI_rel > 1 hypervolume")
    # axs[0,1].legend()
    # plt.show()
    # fig1.savefig(path + str(seed) + "/low_high_FI_hv_vs_combo.svg")

    hv_dict = {"low FI_rel hv": low_FI_hv_list, "high FI_rel hv": high_FI_hv_list, "total hv": total_hv_list}
    hv_df = pd.DataFrame.from_dict(hv_dict)
    # display(hv_df)
    # hv_df.to_csv(path_10000_all+"hypervolume_seed_comparisons.csv")


### Compare top 408 unique circuits from GA run to combinatorial pareto front

In [ ]:
from pymoo.indicators.hv import HV
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
from rankcrowding import RankAndCrowding
ref_point = np.array([0, 0])
hv = HV(ref_point=ref_point)
nds = RankAndCrowding()

path_sc_combo_pareto = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Combinatorial_search/sig_cond_pareto_dsred1.pkl"
path_sc_combo_circuit = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Combinatorial_search/pareto_topo_sigcond_dsred1.pkl"
path_5000 = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_single_cell/synTF-R-DsR/Optimized_hyperparams_fixed_pop_max_hv_ngen50/run7_ngen5000/2025-01-09_Signal_cond_single_DsRED_opt_hps0_ngen5000_seed_"
path_10000 = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Signal_conditioner_single_cell/synTF-R-DsR/Optimized_hyperparams_fixed_pop_max_hv_ngen50/run8_ngen10000/2025-01-14_Signal_cond_single_DsRED_opt_hps0_ngen10000_seed_"
fname_obj = "all_objectives.pkl"
fname_circuits = "all_circuits.pkl"
paths = [path_5000, path_10000]

with open(path_sc_combo_pareto, "rb") as fid:
    sc_combo_pareto = pickle.load(fid)

sc_combo_pareto = abs(sc_combo_pareto)
sc_pareto_low_FI_idx = np.where(sc_combo_pareto[:,1] <= 1.0)
sc_pareto_high_FI_idx = np.where(sc_combo_pareto[:,1] > 1.0)
sc_pareto_low_FI = sc_combo_pareto[sc_pareto_low_FI_idx]
sc_pareto_high_FI = sc_combo_pareto[sc_pareto_high_FI_idx]

rows, cols = sc_combo_pareto.shape

# Defining a data type
dt = {'names':['f{}'.format(i) for i in range(cols)],
       'formats':cols * [sc_combo_pareto.dtype]}

sorted_combo_pareto = sc_combo_pareto[np.argsort(sc_combo_pareto[:, 1])]
sorted_combo_pareto = np.around(sorted_combo_pareto, decimals=9)
# hv_combo = hv(sorted_combo_pareto*-1)
# print("combinatorial hv:", hv_combo)

ref_point2 = np.array([0, -1*max(sc_pareto_low_FI[:, 1])])
hv2 = HV(ref_point=ref_point2)

pareto_low_FI_hv = hv(sc_pareto_low_FI*-1)
pareto_high_FI_hv = hv2(sc_pareto_high_FI*-1)
print("combo low FI hv: ", pareto_low_FI_hv)
print("combo high FI hv: ", pareto_high_FI_hv)

# sorted_combo_circuit = sc_combo_circuit[np.argsort(sc_combo_pareto[:, 1])]
for path in paths:
    ngens = path[-11:-6]
    fig1, axs = plt.subplots(1, 3, figsize=(7, 2))
    xticks = []
    high_FI_hv_list = []
    low_FI_hv_list = []
    total_hv_list = []
    for seed in range(0, 10):
        print("seed: ", seed)
        objs_path = path + str(seed) + "/" + fname_obj
        with open(objs_path, "rb") as fid:
            objs_GA = pickle.load(fid)

        round_objs_GA = np.around(objs_GA, decimals=9)
        unique_objs_GA, idx_unique_GA = np.unique(round_objs_GA, axis=0, return_index=True)
        
        S = nds.do(unique_objs_GA, 408)
        unique_objs_GA = unique_objs_GA[S]
        hv_GA = hv(unique_objs_GA)
        unique_objs_GA = abs(unique_objs_GA)
        sorted_objs_GA = unique_objs_GA[np.argsort(unique_objs_GA[:, 1])]

        low_FI_objs_idx = np.where(unique_objs_GA[:,1] <= 1.0)
        high_FI_objs_idx = np.where(unique_objs_GA[:,1] > 1.0)
        low_FI_objs_arr = unique_objs_GA[low_FI_objs_idx]
        high_FI_objs_arr = unique_objs_GA[high_FI_objs_idx]

        ref_point_2_GA = np.array([0, -1*max(low_FI_objs_arr[:, 1])])
        hv2_GA = HV(ref_point=ref_point_2_GA)

        low_FI_hv_GA = hv(low_FI_objs_arr*-1)
        high_FI_hv_GA = hv2_GA(high_FI_objs_arr*-1)
        high_FI_hv_list.append(high_FI_hv_GA)
        low_FI_hv_list.append(low_FI_hv_GA)
        total_hv_list.append(hv_GA)
        print("GA", ngens, "gens low FI hv:", low_FI_hv_GA)
        print("GA", ngens, "gens high FI hv:", high_FI_hv_GA)
        print("GA", ngens, "gens total hv:", low_FI_hv_GA+high_FI_hv_GA)


        not_in_GA = np.setdiff1d(sorted_combo_pareto.copy().view(dt), sorted_objs_GA.copy().view(dt))
        not_in_GA = not_in_GA.view(sorted_combo_pareto.dtype).reshape(-1, cols)
        high_FI_idx = np.where(not_in_GA[:,1] > 1.0)
        high_FI_not_in_GA = (not_in_GA[high_FI_idx])*-1
        low_FI_idx = np.where(not_in_GA[:,1] < 1.0)
        low_FI_not_in_GA = (not_in_GA[low_FI_idx])*-1
        print("not in GA: ", len(not_in_GA))
        print("number of objectives with FI_rel > 1 that are not in GA top 408:", len(high_FI_not_in_GA))
        print("number of objectives with FI_rel < 1 that are not in GA top 408:", len(low_FI_not_in_GA))

        high_FI_idx = np.where(not_in_GA[:,1] > 1.0)
        high_FI_not_in_GA = (not_in_GA[high_FI_idx])*-1

        not_in_pareto = np.setdiff1d(sorted_objs_GA.copy().view(dt), sorted_combo_pareto.copy().view(dt))
        not_in_pareto = not_in_pareto.view(sorted_combo_pareto.dtype).reshape(-1, cols)
        print("not in pareto: ", len(not_in_pareto))
        high_FI_idx_2 = np.where(not_in_pareto[:,1] > 1.0)
        high_FI_not_in_pareto = (not_in_pareto[high_FI_idx_2])*-1

        fig2 = plt.figure(figsize=(2.5, 2.5))
        plt.plot(sc_combo_pareto[:,0], sc_combo_pareto[:,1], linestyle="none", marker="o", markersize="3", label="combinatorial")
        plt.plot(unique_objs_GA[:,0], unique_objs_GA[:,1], linestyle="none", marker="o", markersize="3", fillstyle='none', label="GA")
        plt.title("Seed "+str(seed))
        plt.legend()
        # plt.show()
        # fig2.savefig(path + str(seed) + "/top_408_objs_vs_combo_.svg")
        print("_______")

        axs[0].bar(seed+1, low_FI_hv_GA, width=0.5, align="center", color="grey")
        axs[1].bar(seed+1, high_FI_hv_GA, width=0.5, align="center", color="grey")
        axs[2].bar(seed+1, hv_GA, width=0.5, align="center", color="grey")
        xticks.append(seed+1)
    axs[0].axhline(y=pareto_low_FI_hv, linestyle="dashed", color="k", label="combinatorial")
    
    axs[0].set_xticks(xticks)
    axs[0].set_xticklabels([str(seed) for seed in range(10)])
    axs[0].set_xlabel("GA seed")
    axs[0].set_ylim([43.5,45])
    axs[0].set_ylabel("hypervolume")
    axs[0].set_title("FI_rel < 1 hypervolume")
    axs[0].legend(loc="upper right")

    axs[1].axhline(y=pareto_high_FI_hv, linestyle="dashed", color="k", label="combinatorial")
    axs[1].set_xticks(xticks)
    axs[1].set_xticklabels([str(seed) for seed in range(10)])
    axs[1].set_xlabel("GA seed")
    axs[1].set_ylim([0,1.5])
    axs[1].set_ylabel("hypervolume")
    axs[1].set_title("FI_rel > 1 hypervolume")

    axs[2].axhline(y=hv(sc_combo_pareto*-1), linestyle="dashed", color="k", label="combinatorial")
    axs[2].set_xticks(xticks)
    axs[2].set_xticklabels([str(seed) for seed in range(10)])
    axs[2].set_xlabel("GA seed")
    axs[2].set_ylim([44.5, 46])
    axs[2].set_ylabel("hypervolume")
    axs[2].set_title("total hypervolume")
    plt.show()

hv_dict = {"low FI_rel hv": low_FI_hv_list, "high FI_rel hv": high_FI_hv_list, "total hv": total_hv_list}
hv_df = pd.DataFrame.from_dict(hv_dict)
# display(hv_df)
# hv_df.to_csv(path_10000_all+"top_408_hv_seed_comparisons.csv")

# # hv combinatorial = 45.88913385050545

### Compare each gen top 200 objs to pareto

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import pickle
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
from rankcrowding import RankAndCrowding
nds = RankAndCrowding()

path_sc_combo_pareto = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Combinatorial_search/sig_cond_pareto_dsred1.pkl"
path_sc_top_objs = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/2025-01-19_Signal_cond_single_DsRED_opt_hps0_ngen120_seed_9/top_objs_all.pkl"

with open(path_sc_combo_pareto, "rb") as fid:
    sc_combo_pareto = pickle.load(fid)

# sorted_combo_pareto = sc_combo_pareto[np.argsort(sc_combo_pareto[:, 1])]
# sorted_combo_pareto = np.around(sorted_combo_pareto, decimals=9)
sc_combo_pareto = np.around(sc_combo_pareto, decimals=9)


# Defining a data type
rows, cols = sc_combo_pareto.shape
dt = {'names':['f{}'.format(i) for i in range(cols)],
       'formats':cols * [sc_combo_pareto.dtype]}

with open(path_sc_top_objs, "rb") as fid:
    top_200 = pickle.load(fid)

# print(len(top_200[0]))

set_pareto_circuits = []
all_GA_pareto_circuits = []
gens = []
for i in range(len(top_200)):
    print("gen: ", i)
    gens.append(i)
    top_objs_gen = top_200[i]
    # print(top_objs_gen)
    # S, rank_dict = nds.do(top_objs_gen, 200, return_rank=True)
    # sorted_top_objs = top_objs_gen[S]

    objs_GA_round = np.around(deepcopy(top_objs_gen), decimals=9)
    sorted_objs_GA = objs_GA_round[np.argsort(top_objs_gen[:, 1])]

    # set intersection of GA and pareto front (no repeated objs)
    GA_pareto_intersect = np.intersect1d(deepcopy(objs_GA_round).view(dt), deepcopy(sc_combo_pareto).view(dt))
    GA_pareto_intersect = GA_pareto_intersect.view(sc_combo_pareto.dtype).reshape(-1, cols)
    sorted_GA_pareto_intersect = GA_pareto_intersect[np.argsort(GA_pareto_intersect[:, 1])]
    # print(sorted_GA_pareto_intersect)
    num_pareto = len(GA_pareto_intersect)
    print("num pareto intersect 1d:", num_pareto)

    # all objs in GA that are on pareto front (includes repeated objs)
    mask = np.isin(deepcopy(objs_GA_round).view(dt), deepcopy(sc_combo_pareto).view(dt))
    all_GA_in_pareto = objs_GA_round[mask.flatten()]
    sorted_GA_in_pareto = all_GA_in_pareto[np.argsort(all_GA_in_pareto[:, 1])]
    print(sorted_GA_in_pareto)
    num_all_pareto_circuits = len(sorted_GA_in_pareto)
    print("num circuits in pareto w repeats", num_all_pareto_circuits)
#     # print(all_GA_in_pareto0)
    print("_____________")
    set_pareto_circuits.append(num_pareto)
    all_GA_pareto_circuits.append(num_all_pareto_circuits)
    
plt.plot(gens, set_pareto_circuits, label="set")
plt.plot(gens, all_GA_pareto_circuits, label="with repeats")
plt.legend()

In [ ]:
import pandas as pd
path = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/2025-01-21_Signal_cond_single_DsRED_opt_hps0_ngen120_seed_0/rank_dicts_all.pkl"

with open(path, "rb") as fid:
    rank_dicts = pickle.load(fid)

# rank dict index is gen-1
rank_df = pd.DataFrame.from_dict(rank_dicts[38]).T
display(rank_df)
rank_df.to_csv("/Users/kdreyer/Documents/Github/GraphGA/GA_results/2025-01-21_Signal_cond_single_DsRED_opt_hps0_ngen120_seed_0/rank_dict_gen39.csv")

In [ ]:
path = "/Users/kdreyer/Documents/Github/GraphGA/GA_results/2025-01-27_Signal_cond_single_DsRED_opt_hps0_ngen10_seed_0/top_circuits_all.pkl"

with open(path, "rb") as fid:
    top_circuits = pickle.load(fid)

# print(len(top_circuits[2]))
top_circuits_gen = top_circuits[0].flatten()
circuit_in_dict = deepcopy(top_circuits_gen[0].in_dict)
circuit_in_dict["dose"] = top_circuits_gen[0].dose
print(circuit_in_dict.items())

In [ ]:
path = "/Users/kdreyer/Library/CloudStorage/OneDrive-NorthwesternUniversity/KatieD_LL/GCAD_Collab/Selected_GA_results_paper/Combinatorial_search/pulse_all_obj_dsred1.pkl"

with open(path, "rb") as fid:
    ranked_topo = pickle.load(fid)

print(len(ranked_topo))